In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
# Configuration
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
style.use("ggplot")
from textwrap import wrap
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from collections import Counter
import string
import nltk
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, Flatten, MaxPooling1D





2024-06-10 15:18:55.970248: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-10 15:18:55.970460: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-10 15:18:56.218776: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-10 15:18:56.877500: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 15:19:03.951325: W tensorflow/comp

In [3]:
import pandas as pd
df = pd.read_csv('/home/yadagiri/Downloads/new_data_hc3.csv')
#df.to_csv('testing.csv', index=False)

# Human is 0 and AI is 1
# Press Ctrl + F9 to run this whole together

In [4]:
df

,text,label
0,"['Basically there are many categories of "" Bes...",0
1,['salt is good for not dying in car crashes an...,0
2,"[""The way it works is that old TV stations got...",0
3,"[""You ca n't just go around assassinating the ...",0
4,['Wanting to kill the shit out of Germans driv...,0
...,...,...
48639,"[""It's not uncommon for blood pressure to fluc...",1
48640,['There are several possible causes of a painl...,1
48641,"[""It is not appropriate for me to recommend a ...",1
48642,['It is not uncommon for people with rheumatoi...,1


In [104]:
# Removing 'id' and 'Category' columns
df = df.drop(columns=['model','source','id'])

In [66]:
df = df.rename(columns={'sentence': 'text','class': 'generated'})

In [105]:
df

,text,label
0,Forza Motorsport is a popular racing game that...,1
1,Buying Virtual Console games for your Nintendo...,1
2,Windows NT 4.0 was a popular operating system ...,1
3,How to Make Perfume\n\nPerfume is a great way ...,1
4,How to Convert Song Lyrics to a Song'\n\nConve...,1
...,...,...
172412,"During the Cold War, the United States was po...",1
172413,"The ""continuity thesis"" is the idea that ther...",1
172414,"In the early Middle Ages, the pagan Norse wer...",1
172415,There are many similarities between the langu...,1


In [5]:
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')

def calculate(raw_df):
  def tokenize_text(text):
      if not isinstance(text, str):
          return []
      tokens = word_tokenize(text)
      return tokens

  def tokenize_sent(text):
      if not isinstance(text, str):
          return []
      sent = sent_tokenize(text)
      sent_sizes = [len(sent.split()) for sent in sent]
      return sent_sizes

  def sent_wala(text):
    if not isinstance(text, str):
          return []
    sent = sent_tokenize(text)
    return sent

  def vocab_for_row(text):
      unique=set(text)
      return len(unique)

  def number_of_sentence(text):
      return len(text)

  def number_of_words(text):
      return len(text)

  raw_df["words"] = raw_df["text"].apply(tokenize_text)

  raw_df["sentences"] = raw_df["text"].apply(sent_wala)

  raw_df["sentence lengths"] = raw_df["text"].apply(tokenize_sent)

  raw_df["vocab"] =raw_df["words"].apply(vocab_for_row)

  raw_df['line count'] = raw_df["sentence lengths"].apply(number_of_sentence)

  raw_df['word count'] = raw_df['words'].apply(number_of_words)

  raw_df['avg line length'] = raw_df["word count"]/raw_df["line count"]

calculate(df)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/yadagiri/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/yadagiri/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Abhi mere paas 2 dataframe hai human_df and ai_df. Iske baad mujhe dono ko alag alag seh word density, average line length, and vocabulary count nikala hai.

In [6]:
def total_voc(df):
  all_words = [word for sublist in df['words'] for word in sublist]
  all_words = set(all_words)
  return len(all_words)


def sum_of_lines(df):
    return df['line count'].sum()

def average_line_len(df):
    sum_line_avg = df['avg line length'].sum()
    return sum_line_avg/ len(df)

def word_density(df):
    wd= 100*total_voc(df)/ (average_line_len(df)*sum_of_lines(df))
    return wd



In [7]:
import spacy
import pandas as pd
nlp = spacy.load("en_core_web_sm")

def count_stop_words(words):
    doc = nlp(" ".join(words))
    stop_word_count = sum(1 for token in doc if token.is_stop)
    return stop_word_count

df["stopwords_count"]=df["words"].apply(count_stop_words)
df["stopwords_per"]=df["words"].apply(count_stop_words)/df["word count"]*100
df

,text,label,words,sentences,sentence lengths,vocab,line count,word count,avg line length,stopwords_count,stopwords_per
0,"['Basically there are many categories of "" Bes...",0,"[[, 'Basically, there, are, many, categories, ...","[['Basically there are many categories of "" Be...","[11, 28, 33, 6, 13, 18, 18, 7, 53, 21, 8, 23, 32]",133,13,279,21.461538,124,44.444444
1,['salt is good for not dying in car crashes an...,0,"[[, 'salt, is, good, for, not, dying, in, car,...",[['salt is good for not dying in car crashes a...,"[19, 22, 16, 39, 29, 33, 17, 10, 20, 15, 16, 2...",219,22,425,19.318182,194,45.647059
2,"[""The way it works is that old TV stations got...",0,"[[, ``, The, way, it, works, is, that, old, TV...","[[""The way it works is that old TV stations go...","[23, 88, 55, 14, 12, 14, 52, 22, 31, 23, 7, 25...",231,15,492,32.800000,200,40.650407
3,"[""You ca n't just go around assassinating the ...",0,"[[, ``, You, ca, n't, just, go, around, assass...","[[""You ca n't just go around assassinating the...","[16, 7, 59, 6, 3, 31, 9, 18, 25, 16, 13]",133,11,211,19.181818,99,46.919431
4,['Wanting to kill the shit out of Germans driv...,0,"[[, 'Wanting, to, kill, the, shit, out, of, Ge...",[['Wanting to kill the shit out of Germans dri...,"[11, 34, 18, 39, 15, 28, 30, 19, 18, 45, 32]",168,11,294,26.727273,139,47.278912
...,...,...,...,...,...,...,...,...,...,...,...
48639,"[""It's not uncommon for blood pressure to fluc...",1,"[[, ``, It, 's, not, uncommon, for, blood, pre...","[[""It's not uncommon for blood pressure to flu...","[23, 36, 32, 26, 28]",92,5,164,32.800000,79,48.170732
48640,['There are several possible causes of a painl...,1,"[[, 'There, are, several, possible, causes, of...",[['There are several possible causes of a pain...,"[16, 19, 21, 28, 13, 23, 30, 16]",105,8,194,24.250000,85,43.814433
48641,"[""It is not appropriate for me to recommend a ...",1,"[[, ``, It, is, not, appropriate, for, me, to,...","[[""It is not appropriate for me to recommend a...","[18, 11, 41, 60, 16]",88,5,164,32.800000,79,48.170732
48642,['It is not uncommon for people with rheumatoi...,1,"[[, 'It, is, not, uncommon, for, people, with,...",[['It is not uncommon for people with rheumato...,"[26, 15, 46, 25, 13]",95,5,142,28.400000,65,45.774648


In [8]:
# this is active and passive voice which i counted


import spacy
import pandas as pd


nlp = spacy.load("en_core_web_sm")


def is_passive(sentence):
    for token in sentence:
        if token.dep_ == "nsubjpass":
            return True
    return False

def count_active(sentences):
    active_count = 0
    passive_count = 0
    for sent in sentences:
        doc = nlp(sent)
        if is_passive(doc):
            passive_count += 1
    active_count=len(sentences)-passive_count
    return active_count


df["active"]=df["sentences"].apply(count_active)
df["passive"]=df["line count"]-df["active"]

df["active_per"]=df["active"]/df["line count"]*100
df["passive_per"]=df["passive"]/df["line count"]*100


Abhi Maine yaha peh POS TAGS nikale hai

In [9]:
import pandas as pd
import nltk
from nltk import word_tokenize, pos_tag
from collections import Counter

def count_nouns_verbs(word_list):
    tags = pos_tag(word_list)
    counts = Counter(tag for word, tag in tags)
    pos_counts = {
        'NOUN': counts.get('NN', 0) + counts.get('NNS', 0) + counts.get('NNP', 0) + counts.get('NNPS', 0),
        'VERB': counts.get('VB', 0) + counts.get('VBD', 0) + counts.get('VBG', 0) + counts.get('VBN', 0) + counts.get('VBP', 0) + counts.get('VBZ', 0),
        'PUNCT': counts.get('.', 0) + counts.get(',', 0) + counts.get(':', 0) + counts.get('(', 0) + counts.get(')', 0) + counts.get('"', 0) + counts.get("''", 0) + counts.get('``', 0) + counts.get('!', 0) + counts.get('?', 0) + counts.get(';', 0) + counts.get('-', 0),
        'DET': counts.get('DT', 0) + counts.get('PDT', 0) + counts.get('WDT', 0),
        'PRON': counts.get('PRP', 0) + counts.get('PRP$', 0),
        'PROPN': counts.get('NNP', 0) + counts.get('NNPS', 0),
        'ADJ': counts.get('JJ', 0) + counts.get('JJR', 0) + counts.get('JJS', 0),
        'AUX': counts.get('MD', 0),
        'ADV': counts.get('RB', 0) + counts.get('RBR', 0) + counts.get('RBS', 0),
        'PART': counts.get('RP', 0),
        'SCONJ': counts.get('IN', 0),
        'NUM': counts.get('CD', 0),
        'X': counts.get('FW', 0),
        'INTJ': counts.get('UH', 0),
        'ADP': counts.get('IN', 0),
        'SYM': counts.get('SYM', 0),
        'SPACE': counts.get('SP', 0),
        'CCONJ': counts.get('CC', 0)
    }
    return pd.Series(pos_counts)

pos_wala_df  = df['words'].apply(lambda x: pd.Series(count_nouns_verbs(x)))

human_copy=df[df['label'] == 0].copy()
ai_copy=df[df['label'] == 0].copy()

df = pd.concat([df, pos_wala_df], axis=1)
df


,text,label,words,sentences,sentence lengths,vocab,line count,word count,avg line length,stopwords_count,...,ADV,PART,SCONJ,NUM,X,INTJ,ADP,SYM,SPACE,CCONJ
0,"['Basically there are many categories of "" Bes...",0,"[[, 'Basically, there, are, many, categories, ...","[['Basically there are many categories of "" Be...","[11, 28, 33, 6, 13, 18, 18, 7, 53, 21, 8, 23, 32]",133,13,279,21.461538,124,...,23,0,24,5,0,0,24,0,0,12
1,['salt is good for not dying in car crashes an...,0,"[[, 'salt, is, good, for, not, dying, in, car,...",[['salt is good for not dying in car crashes a...,"[19, 22, 16, 39, 29, 33, 17, 10, 20, 15, 16, 2...",219,22,425,19.318182,194,...,31,2,48,3,0,3,48,0,0,15
2,"[""The way it works is that old TV stations got...",0,"[[, ``, The, way, it, works, is, that, old, TV...","[[""The way it works is that old TV stations go...","[23, 88, 55, 14, 12, 14, 52, 22, 31, 23, 7, 25...",231,15,492,32.800000,200,...,32,4,50,16,1,0,50,0,0,13
3,"[""You ca n't just go around assassinating the ...",0,"[[, ``, You, ca, n't, just, go, around, assass...","[[""You ca n't just go around assassinating the...","[16, 7, 59, 6, 3, 31, 9, 18, 25, 16, 13]",133,11,211,19.181818,99,...,22,1,16,1,0,0,16,0,0,7
4,['Wanting to kill the shit out of Germans driv...,0,"[[, 'Wanting, to, kill, the, shit, out, of, Ge...",[['Wanting to kill the shit out of Germans dri...,"[11, 34, 18, 39, 15, 28, 30, 19, 18, 45, 32]",168,11,294,26.727273,139,...,23,6,27,4,1,0,27,0,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48639,"[""It's not uncommon for blood pressure to fluc...",1,"[[, ``, It, 's, not, uncommon, for, blood, pre...","[[""It's not uncommon for blood pressure to flu...","[23, 36, 32, 26, 28]",92,5,164,32.800000,79,...,8,0,14,1,0,0,14,0,0,5
48640,['There are several possible causes of a painl...,1,"[[, 'There, are, several, possible, causes, of...",[['There are several possible causes of a pain...,"[16, 19, 21, 28, 13, 23, 30, 16]",105,8,194,24.250000,85,...,7,0,16,0,0,0,16,0,0,9
48641,"[""It is not appropriate for me to recommend a ...",1,"[[, ``, It, is, not, appropriate, for, me, to,...","[[""It is not appropriate for me to recommend a...","[18, 11, 41, 60, 16]",88,5,164,32.800000,79,...,6,0,11,0,0,0,11,0,0,7
48642,['It is not uncommon for people with rheumatoi...,1,"[[, 'It, is, not, uncommon, for, people, with,...",[['It is not uncommon for people with rheumato...,"[26, 15, 46, 25, 13]",95,5,142,28.400000,65,...,5,0,16,3,0,0,16,0,0,9


In [10]:
df.to_csv('/home/yadagiri/Downloads/new_data_hc3.csv', index=False)

In [11]:
human_df = df[df['label'] == 0].reset_index(drop=True)
ai_df = df[df['label'] == 1].reset_index(drop=True)

In [12]:
# for ai pos
ai_word_count=ai_df.iloc[:,7].sum()
pos_sum_ai = ai_df.iloc[:, 15:].sum()
pos_df_ai = pd.DataFrame(pos_sum_ai, columns=['POS_Sum'])
def contribution(val):
  return val*100/ai_word_count
pos_df_ai=pos_df_ai.apply(contribution)

# for human pos
human_word_count=human_df.iloc[:,7].sum()
pos_sum_human = human_df.iloc[:, 15:].sum()
pos_df_human = pd.DataFrame(pos_sum_human, columns=['POS_Sum'])
def contribution(val):
  return val*100/human_word_count
pos_df_human=pos_df_human.apply(contribution)

In [13]:
pos_df_ai
# ai contribution of pos tags

,POS_Sum
NOUN,24.519251
VERB,16.052922
PUNCT,9.724757
DET,10.665753
PRON,4.673576
PROPN,2.527910
ADJ,8.307765
AUX,2.113747
ADV,3.501989
PART,0.255427


In [14]:
pos_df_human
# human contribution of pos tags

,POS_Sum
NOUN,22.688839
VERB,15.647814
PUNCT,13.051546
DET,9.540641
PRON,5.783289
PROPN,3.731803
ADJ,7.242732
AUX,1.557435
ADV,5.373202
PART,0.427550


In [15]:
print(f"For Human :\nAverage Line Length {round(average_line_len(human_df),1)}\nTotal Vocabulary {total_voc(human_df)}\nWord Density {round(word_density(human_df),2)}")

print(f"\nFor AI :\nAverage Line Length {round(average_line_len(ai_df),1)}\nTotal Vocabulary {total_voc(ai_df)}\nWord Density {round(word_density(ai_df),2)}")

For Human :
Average Line Length 23.7
Total Vocabulary 116998
Word Density 1.32

For AI :
Average Line Length 32.4
Total Vocabulary 93476
Word Density 1.7


In [16]:
linking_words = {'to', 'the', 'and', 'of', 'in', 'on', 'for', 'with', 'at', 'a', 'an'}
def count_punctuation_marks(text):
    punctuation_count = sum(1 for char in text if char in string.punctuation)
    return punctuation_count

def count_linking_words(text):
    linking_words_count = sum(1 for word in word_tokenize(text.lower()) if word in linking_words)
    return linking_words_count

df['punctuation_count'] = df['text'].apply(count_punctuation_marks)

df['linking_words_count'] = df['text'].apply(count_linking_words)

df['punc_per']=df['punctuation_count']/df['word count']*100
df['link_per']=df['linking_words_count']/df['word count']*100



print(df)

                                                    text  label  \
0      ['Basically there are many categories of " Bes...      0   
1      ['salt is good for not dying in car crashes an...      0   
2      ["The way it works is that old TV stations got...      0   
3      ["You ca n't just go around assassinating the ...      0   
4      ['Wanting to kill the shit out of Germans driv...      0   
...                                                  ...    ...   
48639  ["It's not uncommon for blood pressure to fluc...      1   
48640  ['There are several possible causes of a painl...      1   
48641  ["It is not appropriate for me to recommend a ...      1   
48642  ['It is not uncommon for people with rheumatoi...      1   
48643  ['It is not uncommon to experience back pain, ...      1   

                                                   words  \
0      [[, 'Basically, there, are, many, categories, ...   
1      [[, 'salt, is, good, for, not, dying, in, car,...   
2      [[, ``, 

In [17]:
ai_df

,text,label,words,sentences,sentence lengths,vocab,line count,word count,avg line length,stopwords_count,...,ADV,PART,SCONJ,NUM,X,INTJ,ADP,SYM,SPACE,CCONJ
0,['There are many different best seller lists t...,1,"[[, 'There, are, many, different, best, seller...",[['There are many different best seller lists ...,"[23, 30, 28, 39, 18, 32, 37]",99,7,228,32.571429,114,...,16,0,24,4,0,0,24,0,0,7
1,"[""Salt is used on roads to help melt ice and s...",1,"[[, ``, Salt, is, used, on, roads, to, help, m...","[[""Salt is used on roads to help melt ice and ...","[18, 23, 30, 29, 22, 31, 23, 23]",119,8,226,28.250000,105,...,8,0,29,0,0,0,29,0,0,15
2,"[""There are a few reasons why we still have SD...",1,"[[, ``, There, are, a, few, reasons, why, we, ...","[[""There are a few reasons why we still have S...","[15, 15, 22, 1, 24, 15, 1, 23, 30]",89,9,174,19.333333,83,...,17,1,11,0,0,0,11,0,0,2
3,['It is generally not acceptable or ethical to...,1,"[[, 'It, is, generally, not, acceptable, or, e...",[['It is generally not acceptable or ethical t...,"[23, 10, 32, 12, 25, 23, 20]",102,7,161,23.000000,81,...,10,1,14,0,0,0,14,0,0,13
4,['After the Wright Brothers made the first pow...,1,"[[, 'After, the, Wright, Brothers, made, the, ...",[['After the Wright Brothers made the first po...,"[23, 19, 20, 17, 25, 18, 24, 35, 18]",107,9,225,25.000000,100,...,13,0,32,1,0,0,32,0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24317,"[""It's not uncommon for blood pressure to fluc...",1,"[[, ``, It, 's, not, uncommon, for, blood, pre...","[[""It's not uncommon for blood pressure to flu...","[23, 36, 32, 26, 28]",92,5,164,32.800000,79,...,8,0,14,1,0,0,14,0,0,5
24318,['There are several possible causes of a painl...,1,"[[, 'There, are, several, possible, causes, of...",[['There are several possible causes of a pain...,"[16, 19, 21, 28, 13, 23, 30, 16]",105,8,194,24.250000,85,...,7,0,16,0,0,0,16,0,0,9
24319,"[""It is not appropriate for me to recommend a ...",1,"[[, ``, It, is, not, appropriate, for, me, to,...","[[""It is not appropriate for me to recommend a...","[18, 11, 41, 60, 16]",88,5,164,32.800000,79,...,6,0,11,0,0,0,11,0,0,7
24320,['It is not uncommon for people with rheumatoi...,1,"[[, 'It, is, not, uncommon, for, people, with,...",[['It is not uncommon for people with rheumato...,"[26, 15, 46, 25, 13]",95,5,142,28.400000,65,...,5,0,16,3,0,0,16,0,0,9


In [18]:
human_df

,text,label,words,sentences,sentence lengths,vocab,line count,word count,avg line length,stopwords_count,...,ADV,PART,SCONJ,NUM,X,INTJ,ADP,SYM,SPACE,CCONJ
0,"['Basically there are many categories of "" Bes...",0,"[[, 'Basically, there, are, many, categories, ...","[['Basically there are many categories of "" Be...","[11, 28, 33, 6, 13, 18, 18, 7, 53, 21, 8, 23, 32]",133,13,279,21.461538,124,...,23,0,24,5,0,0,24,0,0,12
1,['salt is good for not dying in car crashes an...,0,"[[, 'salt, is, good, for, not, dying, in, car,...",[['salt is good for not dying in car crashes a...,"[19, 22, 16, 39, 29, 33, 17, 10, 20, 15, 16, 2...",219,22,425,19.318182,194,...,31,2,48,3,0,3,48,0,0,15
2,"[""The way it works is that old TV stations got...",0,"[[, ``, The, way, it, works, is, that, old, TV...","[[""The way it works is that old TV stations go...","[23, 88, 55, 14, 12, 14, 52, 22, 31, 23, 7, 25...",231,15,492,32.800000,200,...,32,4,50,16,1,0,50,0,0,13
3,"[""You ca n't just go around assassinating the ...",0,"[[, ``, You, ca, n't, just, go, around, assass...","[[""You ca n't just go around assassinating the...","[16, 7, 59, 6, 3, 31, 9, 18, 25, 16, 13]",133,11,211,19.181818,99,...,22,1,16,1,0,0,16,0,0,7
4,['Wanting to kill the shit out of Germans driv...,0,"[[, 'Wanting, to, kill, the, shit, out, of, Ge...",[['Wanting to kill the shit out of Germans dri...,"[11, 34, 18, 39, 15, 28, 30, 19, 18, 45, 32]",168,11,294,26.727273,139,...,23,6,27,4,1,0,27,0,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24317,['Hello!Welcome and thank you for asking on HC...,0,"[[, 'Hello, !, Welcome, and, thank, you, for, ...",[['Hello!Welcome and thank you for asking on H...,"[24, 8, 10, 27, 26, 37, 10]",109,7,165,23.571429,62,...,4,0,15,1,0,0,15,0,0,4
24318,"['Hi, * As per my surgical experience, the iss...",0,"[[, 'Hi, ,, *, As, per, my, surgical, experien...","[['Hi, * As per my surgical experience, the is...","[22, 26, 6, 9, 8]",71,5,84,16.800000,33,...,1,1,10,1,0,0,10,0,0,2
24319,['Although it is difficult to comment whether ...,0,"[[, 'Although, it, is, difficult, to, comment,...",[['Although it is difficult to comment whether...,"[22, 9, 27, 21]",66,4,90,22.500000,43,...,7,0,9,0,0,0,9,0,0,3
24320,['Welcome and thank you for asking on HCM! I h...,0,"[[, 'Welcome, and, thank, you, for, asking, on...","[['Welcome and thank you for asking on HCM!, I...","[8, 6, 27]",48,3,51,17.000000,18,...,3,0,4,1,0,0,4,0,0,2


In [19]:
ai_copy

,text,label,words,sentences,sentence lengths,vocab,line count,word count,avg line length,stopwords_count,stopwords_per,active,passive,active_per,passive_per
0,"['Basically there are many categories of "" Bes...",0,"[[, 'Basically, there, are, many, categories, ...","[['Basically there are many categories of "" Be...","[11, 28, 33, 6, 13, 18, 18, 7, 53, 21, 8, 23, 32]",133,13,279,21.461538,124,44.444444,13,0,100.000000,0.000000
1,['salt is good for not dying in car crashes an...,0,"[[, 'salt, is, good, for, not, dying, in, car,...",[['salt is good for not dying in car crashes a...,"[19, 22, 16, 39, 29, 33, 17, 10, 20, 15, 16, 2...",219,22,425,19.318182,194,45.647059,17,5,77.272727,22.727273
2,"[""The way it works is that old TV stations got...",0,"[[, ``, The, way, it, works, is, that, old, TV...","[[""The way it works is that old TV stations go...","[23, 88, 55, 14, 12, 14, 52, 22, 31, 23, 7, 25...",231,15,492,32.800000,200,40.650407,11,4,73.333333,26.666667
3,"[""You ca n't just go around assassinating the ...",0,"[[, ``, You, ca, n't, just, go, around, assass...","[[""You ca n't just go around assassinating the...","[16, 7, 59, 6, 3, 31, 9, 18, 25, 16, 13]",133,11,211,19.181818,99,46.919431,10,1,90.909091,9.090909
4,['Wanting to kill the shit out of Germans driv...,0,"[[, 'Wanting, to, kill, the, shit, out, of, Ge...",[['Wanting to kill the shit out of Germans dri...,"[11, 34, 18, 39, 15, 28, 30, 19, 18, 45, 32]",168,11,294,26.727273,139,47.278912,8,3,72.727273,27.272727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24317,['Hello!Welcome and thank you for asking on HC...,0,"[[, 'Hello, !, Welcome, and, thank, you, for, ...",[['Hello!Welcome and thank you for asking on H...,"[24, 8, 10, 27, 26, 37, 10]",109,7,165,23.571429,62,37.575758,5,2,71.428571,28.571429
24318,"['Hi, * As per my surgical experience, the iss...",0,"[[, 'Hi, ,, *, As, per, my, surgical, experien...","[['Hi, * As per my surgical experience, the is...","[22, 26, 6, 9, 8]",71,5,84,16.800000,33,39.285714,5,0,100.000000,0.000000
24319,['Although it is difficult to comment whether ...,0,"[[, 'Although, it, is, difficult, to, comment,...",[['Although it is difficult to comment whether...,"[22, 9, 27, 21]",66,4,90,22.500000,43,47.777778,4,0,100.000000,0.000000
24320,['Welcome and thank you for asking on HCM! I h...,0,"[[, 'Welcome, and, thank, you, for, asking, on...","[['Welcome and thank you for asking on HCM!, I...","[8, 6, 27]",48,3,51,17.000000,18,35.294118,2,1,66.666667,33.333333


In [20]:
human_copy

,text,label,words,sentences,sentence lengths,vocab,line count,word count,avg line length,stopwords_count,stopwords_per,active,passive,active_per,passive_per
0,"['Basically there are many categories of "" Bes...",0,"[[, 'Basically, there, are, many, categories, ...","[['Basically there are many categories of "" Be...","[11, 28, 33, 6, 13, 18, 18, 7, 53, 21, 8, 23, 32]",133,13,279,21.461538,124,44.444444,13,0,100.000000,0.000000
1,['salt is good for not dying in car crashes an...,0,"[[, 'salt, is, good, for, not, dying, in, car,...",[['salt is good for not dying in car crashes a...,"[19, 22, 16, 39, 29, 33, 17, 10, 20, 15, 16, 2...",219,22,425,19.318182,194,45.647059,17,5,77.272727,22.727273
2,"[""The way it works is that old TV stations got...",0,"[[, ``, The, way, it, works, is, that, old, TV...","[[""The way it works is that old TV stations go...","[23, 88, 55, 14, 12, 14, 52, 22, 31, 23, 7, 25...",231,15,492,32.800000,200,40.650407,11,4,73.333333,26.666667
3,"[""You ca n't just go around assassinating the ...",0,"[[, ``, You, ca, n't, just, go, around, assass...","[[""You ca n't just go around assassinating the...","[16, 7, 59, 6, 3, 31, 9, 18, 25, 16, 13]",133,11,211,19.181818,99,46.919431,10,1,90.909091,9.090909
4,['Wanting to kill the shit out of Germans driv...,0,"[[, 'Wanting, to, kill, the, shit, out, of, Ge...",[['Wanting to kill the shit out of Germans dri...,"[11, 34, 18, 39, 15, 28, 30, 19, 18, 45, 32]",168,11,294,26.727273,139,47.278912,8,3,72.727273,27.272727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24317,['Hello!Welcome and thank you for asking on HC...,0,"[[, 'Hello, !, Welcome, and, thank, you, for, ...",[['Hello!Welcome and thank you for asking on H...,"[24, 8, 10, 27, 26, 37, 10]",109,7,165,23.571429,62,37.575758,5,2,71.428571,28.571429
24318,"['Hi, * As per my surgical experience, the iss...",0,"[[, 'Hi, ,, *, As, per, my, surgical, experien...","[['Hi, * As per my surgical experience, the is...","[22, 26, 6, 9, 8]",71,5,84,16.800000,33,39.285714,5,0,100.000000,0.000000
24319,['Although it is difficult to comment whether ...,0,"[[, 'Although, it, is, difficult, to, comment,...",[['Although it is difficult to comment whether...,"[22, 9, 27, 21]",66,4,90,22.500000,43,47.777778,4,0,100.000000,0.000000
24320,['Welcome and thank you for asking on HCM! I h...,0,"[[, 'Welcome, and, thank, you, for, asking, on...","[['Welcome and thank you for asking on HCM!, I...","[8, 6, 27]",48,3,51,17.000000,18,35.294118,2,1,66.666667,33.333333


In [21]:

human_stopwords_avg=round(human_df["stopwords_count"].sum()/df["word count"].sum()*100,2)
ai_stopwords_avg=round(ai_df["stopwords_count"].sum()/df["word count"].sum()*100,2)
print(f"Human Stopwords Average : {human_stopwords_avg}     AI Stopwords Average : {ai_stopwords_avg}")

Human Stopwords Average : 28.49     AI Stopwords Average : 18.84


In [22]:

human_active_avg=round(human_df["active"].sum()/human_df["line count"].sum()*100,2)
human_passive_avg=round(human_df["passive"].sum()/human_df["line count"].sum()*100,2)

ai_active_avg=round(ai_df["active"].sum()/ai_df["line count"].sum()*100,2)
ai_passive_avg=round(ai_df["passive"].sum()/ai_df["line count"].sum()*100,2)

print(f"Human Active Average : {human_active_avg}\nHuman Passive Average : {human_passive_avg}")
print(f"\nAI Active Average : {ai_active_avg}\nAI Passive Average : {ai_passive_avg}")

Human Active Average : 85.27
Human Passive Average : 14.73

AI Active Average : 71.43
AI Passive Average : 28.57


In [23]:
df['word density']=100*(df['vocab'])/(df['line count']*df['avg line length'])
df

,text,label,words,sentences,sentence lengths,vocab,line count,word count,avg line length,stopwords_count,...,INTJ,ADP,SYM,SPACE,CCONJ,punctuation_count,linking_words_count,punc_per,link_per,word density
0,"['Basically there are many categories of "" Bes...",0,"[[, 'Basically, there, are, many, categories, ...","[['Basically there are many categories of "" Be...","[11, 28, 33, 6, 13, 18, 18, 7, 53, 21, 8, 23, 32]",133,13,279,21.461538,124,...,0,24,0,0,12,68,36,24.372760,12.903226,47.670251
1,['salt is good for not dying in car crashes an...,0,"[[, 'salt, is, good, for, not, dying, in, car,...",[['salt is good for not dying in car crashes a...,"[19, 22, 16, 39, 29, 33, 17, 10, 20, 15, 16, 2...",219,22,425,19.318182,194,...,3,48,0,0,15,65,67,15.294118,15.764706,51.529412
2,"[""The way it works is that old TV stations got...",0,"[[, ``, The, way, it, works, is, that, old, TV...","[[""The way it works is that old TV stations go...","[23, 88, 55, 14, 12, 14, 52, 22, 31, 23, 7, 25...",231,15,492,32.800000,200,...,0,50,0,0,13,75,69,15.243902,14.024390,46.951220
3,"[""You ca n't just go around assassinating the ...",0,"[[, ``, You, ca, n't, just, go, around, assass...","[[""You ca n't just go around assassinating the...","[16, 7, 59, 6, 3, 31, 9, 18, 25, 16, 13]",133,11,211,19.181818,99,...,0,16,0,0,7,46,23,21.800948,10.900474,63.033175
4,['Wanting to kill the shit out of Germans driv...,0,"[[, 'Wanting, to, kill, the, shit, out, of, Ge...",[['Wanting to kill the shit out of Germans dri...,"[11, 34, 18, 39, 15, 28, 30, 19, 18, 45, 32]",168,11,294,26.727273,139,...,0,27,0,0,9,41,63,13.945578,21.428571,57.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48639,"[""It's not uncommon for blood pressure to fluc...",1,"[[, ``, It, 's, not, uncommon, for, blood, pre...","[[""It's not uncommon for blood pressure to flu...","[23, 36, 32, 26, 28]",92,5,164,32.800000,79,...,0,14,0,0,5,19,26,11.585366,15.853659,56.097561
48640,['There are several possible causes of a painl...,1,"[[, 'There, are, several, possible, causes, of...",[['There are several possible causes of a pain...,"[16, 19, 21, 28, 13, 23, 30, 16]",105,8,194,24.250000,85,...,0,16,0,0,9,46,44,23.711340,22.680412,54.123711
48641,"[""It is not appropriate for me to recommend a ...",1,"[[, ``, It, is, not, appropriate, for, me, to,...","[[""It is not appropriate for me to recommend a...","[18, 11, 41, 60, 16]",88,5,164,32.800000,79,...,0,11,0,0,7,24,33,14.634146,20.121951,53.658537
48642,['It is not uncommon for people with rheumatoi...,1,"[[, 'It, is, not, uncommon, for, people, with,...",[['It is not uncommon for people with rheumato...,"[26, 15, 46, 25, 13]",95,5,142,28.400000,65,...,0,16,0,0,9,22,29,15.492958,20.422535,66.901408


In [24]:
df.to_csv('/home/yadagiri/Downloads/new_data_hc3_finaldata.csv', index=False)

In [25]:

X = pd.DataFrame(df['word density'])
X['punc_per'] = df['punc_per']
X['link_per'] = df['link_per']

X['avg line length'] = df['avg line length']
X['vocab'] = df['vocab']
X['active_per'] = df['active_per']
X['passive_per'] = df['passive_per']
X['stopwords_per'] = df['stopwords_per']

X['NOUN'] = df['NOUN']
X['VERB'] = df['VERB']
X['PUNCT'] = df['PUNCT']
X['DET'] = df['DET']
X['PRON'] = df['PRON']
X['ADJ'] = df['ADJ']
X['AUX'] = df['AUX']
X['ADV'] = df['ADV']
X['PART'] = df['PART']
X['SCONJ'] = df['SCONJ']
X['NUM'] = df['NUM']
X['X'] = df['X']
X['INTJ'] = df['INTJ']
X['ADP'] = df['ADP']
X['SYM'] = df['SYM']
X['SPACE'] = df['SPACE']
X['CCONJ'] = df['CCONJ']

y = df['generated']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

imputer = SimpleImputer(strategy='constant', fill_value=0)
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)
model = LogisticRegression(max_iter=2000, random_state=42)
model.fit(X_train_imputed, y_train)

y_pred = model.predict(X_test_imputed)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
train_loss = log_loss(y_train, model.predict_proba(X_train_imputed))
test_loss = log_loss(y_test, model.predict_proba(X_test_imputed))
print("Train Loss:", train_loss)
print("Test Loss:", test_loss)

class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)

class_report_df = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose()
print("Class-wise Metrics:")
print(class_report_df)

loss_values = [train_loss, test_loss]
labels = ['Training Loss', 'Testing Loss']

plt.figure(figsize=(8, 6))
plt.bar(labels, loss_values, color=['blue', 'orange'])
plt.title('Logistic Loss')
plt.ylabel('Loss')
plt.show()

KeyError: 'generated'

In [ ]:
X = pd.DataFrame(df['word density'])
X['punc_per'] = df['punc_per']
X['link_per'] = df['link_per']

X['avg line length'] = df['avg line length']
X['vocab'] = df['vocab']
X['active_per'] = df['active_per']
X['passive_per'] = df['passive_per']
X['stopwords_per'] = df['stopwords_per']

X['NOUN'] = df['NOUN']
X['VERB'] = df['VERB']
X['PUNCT'] = df['PUNCT']
X['DET'] = df['DET']
X['PRON'] = df['PRON']
X['ADJ'] = df['ADJ']
X['AUX'] = df['AUX']
X['ADV'] = df['ADV']
X['PART'] = df['PART']
X['SCONJ'] = df['SCONJ']
X['NUM'] = df['NUM']
X['X'] = df['X']
X['INTJ'] = df['INTJ']
X['ADP'] = df['ADP']
X['SYM'] = df['SYM']
X['SPACE'] = df['SPACE']
X['CCONJ'] = df['CCONJ']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

imputer = SimpleImputer(strategy='constant', fill_value=0)

X_train_imputed = imputer.fit_transform(X_train)
X_valid_imputed = imputer.transform(X_valid)
X_train_cnn = X_train_imputed.reshape(X_train_imputed.shape[0], X_train_imputed.shape[1], 1)
X_valid_cnn = X_valid_imputed.reshape(X_valid_imputed.shape[0], X_valid_imputed.shape[1], 1)

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train_cnn, y_train, epochs=10, batch_size=32, validation_data=(X_valid_cnn, y_valid))

y_train_pred_probs = model.predict(X_train_cnn)
y_valid_pred_probs = model.predict(X_valid_cnn)

y_train_pred = (y_train_pred_probs > 0.5).astype(int)
y_valid_pred = (y_valid_pred_probs > 0.5).astype(int)

train_error = 1 - accuracy_score(y_train, y_train_pred)

validation_error = 1 - accuracy_score(y_valid, y_valid_pred)
a
print("Train Error:", train_error)
print("Validation Error:", validation_error)

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['human', 'AI'], yticklabels=['human', 'AI'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()